In [8]:
import pennylane as qml
import numpy as np

def meyer_wallach_q(state, num_qubits):
    """Calculate Meyer-Wallach Q measure for a given state."""
    sum_purities = 0.0
    state = state.reshape([2] * num_qubits)  # Reshape statevector
    
    for qubit in range(num_qubits):
        # Move the target qubit to first position and reshape
        psi = np.moveaxis(state, qubit, 0).reshape(2, -1)
        
        # Calculate reduced density matrix
        rho = psi @ psi.conj().T
        purity = np.trace(rho @ rho).real
        sum_purities += purity
    
    Q = 2 * (1 - sum_purities / num_qubits)
    return Q

# Define a specific circuit (using circuit_2 as an example)
def example_circuit(params, wires):
        qml.Hadamard(wires=wires[0])
        qml.Hadamard(wires=wires[1])
        qml.CZ(wires=[wires[0], wires[1]])
        qml.RX(params[0], wires=wires[0])
        qml.RX(params[1], wires=wires[1])

# Parameters for the circuit
params = np.random.uniform(0, 2 * np.pi, 2)  # change the parameters as needed
num_qubits = 2 # Number of qubits in the circuit (change if depend on your circuit)

# Create device and quantum node
dev = qml.device("default.qubit", wires=num_qubits)

@qml.qnode(dev)
def run_circuit(params, input_state):
    """Run the quantum circuit with given parameters and input state."""
    qml.BasisState(input_state, wires=range(num_qubits))
    example_circuit(params, wires=range(num_qubits))
    return qml.state()

# Calculate Q for all basis states
q_values = []
for state_idx in range(2**num_qubits):
    input_state = [int(x) for x in np.binary_repr(state_idx, width=num_qubits)]
    state = run_circuit(params, input_state)
    Q = meyer_wallach_q(state, num_qubits)
    q_values.append(Q)

# Average Q over all basis states
average_Q = np.mean(q_values)

print(f"Parameters: {params}")
print(f"Q values for each basis state: {q_values}")
print(f"Average Meyer-Wallach Q: {average_Q:.4f}")

Parameters: [5.26436593 3.26552825]
Q values for each basis state: [1.0000000000000009, 1.0000000000000009, 1.0000000000000009, 1.0000000000000009]
Average Meyer-Wallach Q: 1.0000
